In [1]:
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# function to load images
def load_images(folder, label, size):
    images, labels = [], []
    for file in os.listdir(folder):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            path = os.path.join(folder, file)
            try:
                img = Image.open(path).resize((size, size)).convert('RGB')
                images.append(np.array(img))
                labels.append(label)
            except Exception as e:
                print(f"Skipped {file}: {e}")
    return images, labels

# Function to create hdf5 files from folder
def create_hdf5_from_folders(root_folder, output_file='data.h5', image_size=64):
    X_train, Y_train, X_test, Y_test = [], [], [], []

    # Training set
    train_humans_path = os.path.join(root_folder, "training_set/training_set/humans")
    train_nonhumans_path = os.path.join(root_folder, "training_set/training_set/non-humans")

    x, y = load_images(train_humans_path, 1, image_size)
    X_train.extend(x)
    Y_train.extend(y)

    x, y = load_images(train_nonhumans_path, 0, image_size)
    X_train.extend(x)
    Y_train.extend(y)

    # Test set
    test_humans_path = os.path.join(root_folder, "test_set/test_set/humans")
    test_nonhumans_path = os.path.join(root_folder, "test_set/test_set/non-humans")

    x, y = load_images(test_humans_path, 1, image_size)
    X_test.extend(x)
    Y_test.extend(y)

    x, y = load_images(test_nonhumans_path, 0, image_size)
    X_test.extend(x)
    Y_test.extend(y)

    # Convert to NumPy arrays
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)

    # Save to HDF5
    with h5py.File(output_file, 'w') as f:
        f.create_dataset('train_set_x', data=X_train)
        f.create_dataset('train_set_y', data=Y_train)
        f.create_dataset('test_set_x', data=X_test)
        f.create_dataset('test_set_y', data=Y_test)

    print(f"Saved to {output_file} with {X_train.shape[0]} training and {X_test.shape[0]} test images.")

# Example usage
create_hdf5_from_folders("human-and-non-human", output_file='data.h5', image_size=64)

Saved to data.h5 with 8017 training and 326 test images.


In [3]:
def load_dataset():
    dataset = h5py.File("data.h5", "r")
    X_train = np.array(dataset["train_set_x"][:])
    Y_train = np.array(dataset["train_set_y"][:])
    X_test = np.array(dataset["test_set_x"][:])
    Y_test = np.array(dataset["test_set_y"][:])

    Y_train = Y_train.reshape((1, Y_train.shape[0]))
    Y_test = Y_test.reshape((1, Y_test.shape[0]))

    return X_train, Y_train, X_test, Y_test

X_train, Y_train, X_test, Y_test = load_dataset()

In [4]:
X_train = X_train / 255.  # Normalize
X_test = X_test / 255.

X_train = X_train.reshape(X_train.shape[0], -1).T
X_test = X_test.reshape(X_test.shape[0], -1).T

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(1)
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    return {"W1": W1, "b1": b1, "W2": W2, "b2": b2}

def forward_propagation(X, parameters):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    cache = (Z1, A1, Z2, A2)
    return A2, cache

def compute_cost(A2, Y):
    m = Y.shape[1]
    cost = -np.sum(Y * np.log(A2) + (1 - Y) * np.log(1 - A2)) / m
    return np.squeeze(cost)

def backward_propagation(parameters, cache, X, Y):
    m = X.shape[1]
    W2 = parameters['W2']
    
    Z1, A1, Z2, A2 = cache
    dZ2 = A2 - Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return grads

def update_parameters(parameters, grads, learning_rate=0.01):
    parameters['W1'] -= learning_rate * grads['dW1']
    parameters['b1'] -= learning_rate * grads['db1']
    parameters['W2'] -= learning_rate * grads['dW2']
    parameters['b2'] -= learning_rate * grads['db2']
    return parameters


In [6]:
def model(X, Y, n_h, num_iterations=1000, learning_rate=0.01, print_cost=False):
    np.random.seed(3)
    n_x = X.shape[0]
    n_y = Y.shape[0]
    
    parameters = initialize_parameters(n_x, n_h, n_y)

    for i in range(num_iterations):
        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y)
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if print_cost and i % 100 == 0:
            print(f"Cost after iteration {i}: {cost:.4f}")

    return parameters


In [7]:
def predict(parameters, X):
    A2, _ = forward_propagation(X, parameters)
    return (A2 > 0.5).astype(int)

parameters = model(X_train, Y_train, n_h=7, num_iterations=2000, learning_rate=0.005, print_cost=True)

train_preds = predict(parameters, X_train)
test_preds = predict(parameters, X_test)

print("Train Accuracy:", 100 - np.mean(np.abs(train_preds - Y_train)) * 100, "%")
print("Test Accuracy:", 100 - np.mean(np.abs(test_preds - Y_test)) * 100, "%")

Cost after iteration 0: 0.6931
Cost after iteration 100: 0.6715
Cost after iteration 200: 0.5976
Cost after iteration 300: 0.5123
Cost after iteration 400: 0.4332
Cost after iteration 500: 0.3657
Cost after iteration 600: 0.3134
Cost after iteration 700: 0.2734
Cost after iteration 800: 0.2425
Cost after iteration 900: 0.2180
Cost after iteration 1000: 0.1984
Cost after iteration 1100: 0.1824
Cost after iteration 1200: 0.1691
Cost after iteration 1300: 0.1580
Cost after iteration 1400: 0.1485
Cost after iteration 1500: 0.1404
Cost after iteration 1600: 0.1334
Cost after iteration 1700: 0.1273
Cost after iteration 1800: 0.1220
Cost after iteration 1900: 0.1172
Train Accuracy: 96.85667955594361 %
Test Accuracy: 91.41104294478528 %
